<a href="https://colab.research.google.com/github/shinybell/rag-experiment/blob/main/rag_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/shinybell/rag-experiment.git

fatal: destination path 'rag-experiment' already exists and is not an empty directory.


In [ ]:
!cd rag-experiment && git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5 (delta 2), reused 5 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 1.04 KiB | 532.00 KiB/s, done.
From https://github.com/shinybell/rag-experiment
   e699345..cc02dd1  main       -> origin/main
Updating e699345..cc02dd1
Fast-forward
 HOMEWORK.md            | 2 ++
 data/2025questions.csv | 5 +++++
 2 files changed, 7 insertions(+)


In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [ ]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
import pandas as pd
question_df = pd.read_csv("/content/rag-experiment/data/2025questions.csv")
question_df.head()



,Id,Question,Answer
0,0,2025年1月1日にアメリカのニューオーリンズで発生した事件について、犯人の国籍、使用された...,2025年1月1日にアメリカ合衆国ルイジアナ州ニューオーリンズで発生した事件の犯人は42歳の...
1,1,2025年1月にカナダのジャスティン・トルドー首相が辞任を表明した理由は何ですか？,カナダのジャスティン・トルドー首相が2025年1月6日に辞任を表明した理由は、インフレ対応に...
2,2,2025年4月4日に韓国で憲法裁判所により罷免された大統領は誰ですか？,2025年4月4日に韓国で憲法裁判所により罷免された大統領は尹錫悦（ユン・ソンニョル）氏です。
3,3,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画はどのように変更さ...,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画は多額の投資へと変...
4,4,オーストリアのカール・ネーハマー首相が辞任を表明したのはいつですか？,オーストリアのカール・ネーハマー首相が辞任を表明したのは2025年1月4日です。


In [ ]:
responses = []
for index, row in question_df.iterrows():
    question = row['Question']
    response = generate_output(question)
    responses.append(response)

In [ ]:
# responsesをdataframeに追加
result_df = question_df.copy()
result_df['withoutRAG'] = responses
result_df.head()

,Id,Question,Answer,withoutRAG
0,0,2025年1月1日にアメリカのニューオーリンズで発生した事件について、犯人の国籍、使用された...,2025年1月1日にアメリカ合衆国ルイジアナ州ニューオーリンズで発生した事件の犯人は42歳の...,申し訳ありませんが、2025年1月1日にニューオーリンズやラスベガスで発生した事件に関する情...
1,1,2025年1月にカナダのジャスティン・トルドー首相が辞任を表明した理由は何ですか？,カナダのジャスティン・トルドー首相が2025年1月6日に辞任を表明した理由は、インフレ対応に...,2025年1月にカナダのジャスティン・トルドー首相が辞任を表明した理由は、**複雑な政治的状...
2,2,2025年4月4日に韓国で憲法裁判所により罷免された大統領は誰ですか？,2025年4月4日に韓国で憲法裁判所により罷免された大統領は尹錫悦（ユン・ソンニョル）氏です。,2025年4月4日に韓国で憲法裁判所により罷免された大統領は **尹錫悦** です。 \n\...
3,3,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画はどのように変更さ...,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画は多額の投資へと変...,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画は、**当初の計...
4,4,オーストリアのカール・ネーハマー首相が辞任を表明したのはいつですか？,オーストリアのカール・ネーハマー首相が辞任を表明したのは2025年1月4日です。,オーストリアのカール・ネーハマー首相が辞任を表明したのは **2023年10月17日** で...


- 数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。

- 今回はgemmaでは性能が不安定だったので、OpenAIのgpt-4oで評価していきます。従って、scoreの実行はopenAI APIキーを所持している関心がある方のみで良いです。

In [ ]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.3/661.3 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [ ]:
# @title 評価実装
# gold_answer = "「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。"

# from openai import OpenAI
# from google.colab import userdata
# client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

# def openai_generator(query):

#         messages = [
#                     {
#                         "role": "user",
#                         "content": query
#                     }
#                 ]

#         response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages
#         )
#         return response.choices[0].message.content

# gemini

# def evaluate_answer_accuracy(query, response, reference):

#     template_accuracy1 = (
#           "Instruction: You are a world class state of the art assistant for rating "
#           "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
#           "Say 4, if User Answer is full contained and equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
#           "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
#           "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
#           "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
#           "### Question: {query}\n"
#           "### {answer0}: {sentence_inference}\n"
#           "### {answer1}: {sentence_true}\n"
#           "The rating is:\n"
#       )
#     template_accuracy2 = (
#           "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
#           "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
#           "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
#           "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
#           "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
#           "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
#           "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
#           "Question: {query}\n\n"
#           "{answer0}: {sentence_inference}\n\n"
#           "{answer1}: {sentence_true}\n\n"
#           "Rating: "
#       )

#     score1 = openai_generator(
#                 template_accuracy1.format(
#                       query=query,
#                       answer0="User Answer",
#                       answer1="Reference Answer",
#                       sentence_inference=response,
#                       sentence_true=reference,
#                     )
#                 )
#     try:
#       score1 = int(score1)
#     except:
#       print("Failed")
#       score1 = 0

#     score2 = openai_generator(
#                 template_accuracy2.format(
#                         query=query,
#                         answer0="Reference Answer",
#                         answer1="User Answer",
#                         sentence_inference=reference,
#                         sentence_true=response,
#                     )
#                   )

#     try:
#       score2 = int(score2)
#     except:
#       print("Failed")
#       score2 = 0


#     return (score1 + score2) / 2

In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

0.0


## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用

## 2.1 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
with open("/content/rag-experiment/data/2025.txt", "r") as f:
  raw_txt = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_txt.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[0])

ドキュメントサイズ:  150
ドキュメントの例: 
 1月
1月1日
アメリカ合衆国ルイジアナ州ニューオーリンズで、新年を祝う群衆にピックアップトラック「フォード・F-150 ライトニング」が突っ込み、その後犯人が発砲する事件が発生し、15人が死亡し約30人が負傷した


In [ ]:
# Retrievalの実行
question = question_df.iloc[0]["Question"]

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[83.94237518310547, 73.02055358886719, 69.4022445678711, 76.07028198242188, 75.08028411865234, 70.61888885498047, 68.75859832763672, 63.844425201416016, 65.599365234375, 65.65267181396484, 66.21966552734375, 66.69340515136719, 65.54029846191406, 67.3956298828125, 66.70683288574219, 66.61241912841797, 64.89354705810547, 66.01719665527344, 66.14129638671875, 63.8702392578125, 69.03702545166016, 62.541603088378906, 67.4932632446289, 68.56598663330078, 69.37445068359375, 66.78640747070312, 65.05298614501953, 69.68859100341797, 68.39720916748047, 65.89592742919922, 68.8908920288086, 63.99077224731445, 61.6238899230957, 66.36293029785156, 66.81654357910156, 64.88611602783203, 70.92604064941406, 66.46546936035156, 68.34835815429688, 65.39817810058594, 63.39312744140625, 67.778076171875, 70.2983627319336, 68.29235076904297, 68.16326141357422, 66.6137924194336, 66.9072036743164, 65.87480163574219, 71.61576080322266, 70.29624938964844, 66.95043182373047, 67.11153411865234, 69.82479858398438, 69

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 83.94237518310547)
1月
1月1日
アメリカ合衆国ルイジアナ州ニューオーリンズで、新年を祝う群衆にピックアップトラック「フォード・F-150 ライトニング」が突っ込み、その後犯人が発砲する事件が発生し、15人が死亡し約30人が負傷した 


取得したドキュメント2: (Score: 76.07028198242188)
アメリカ合衆国ネバダ州ラスベガスのトランプ・インターナショナル・ホテル・ラスベガスの玄関車寄せ付近に於いて、ピックアップトラック「テスラ・サイバートラック」が爆発炎上し、運転手と思われる人物が死亡し、付近に居た通行人ら7名が負傷する事件が発生した 


取得したドキュメント3: (Score: 75.18238067626953)
2月15日
オーストリアの都市フィラッハで、23歳のシリア人が居合わせた歩行者6人に対して無差別殺傷事件を起こす 


取得したドキュメント4: (Score: 75.08028411865234)
尚、アメリカ連邦捜査局（FBI）はニューオーリンズの事件とラスベガスの事件に関連性があるかどうかについて捜査を行う方針 


取得したドキュメント5: (Score: 73.02055358886719)
犯人は、42歳のアメリカ国籍の男で、警察官に向かって発砲し、銃撃戦により死亡した 




In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] {question_df.iloc[0]['Question']}"
response = generate_output(query)
print(response)

## 2025年1月1日アメリカのニューオーリンズ・ラスベガス事件について

**ニューオーリンズ事件**

* **犯人の国籍:**  アメリカ国籍
* **使用された車の車種:**  フォード・F-150 ライトニング
* **死傷者数:** 15人死亡、約30人負傷

**ラスベガス事件**

* **使用された車の車種:** テスラ・サイバートラック


 





In [ ]:
rag_responses = []
queries = []
for index, row in question_df.iterrows():
    question = row['Question']
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(documents)
    scores = (query_embeddings @ document_embeddings.T) * 100
    topk = 5
    for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
      print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
      print(documents[index], "\n\n")
    references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
    query =  f"[参考資料]\n{references}\n\n[質問] {question}"
    queries.append(query)
    response = generate_output(query)
    rag_responses.append(response)

result_df['Query'] = queries
result_df['withRAG'] = rag_responses
result_df.head()

取得したドキュメント1: (Score: 83.94237518310547)
1月
1月1日
アメリカ合衆国ルイジアナ州ニューオーリンズで、新年を祝う群衆にピックアップトラック「フォード・F-150 ライトニング」が突っ込み、その後犯人が発砲する事件が発生し、15人が死亡し約30人が負傷した 


取得したドキュメント2: (Score: 76.07028198242188)
アメリカ合衆国ネバダ州ラスベガスのトランプ・インターナショナル・ホテル・ラスベガスの玄関車寄せ付近に於いて、ピックアップトラック「テスラ・サイバートラック」が爆発炎上し、運転手と思われる人物が死亡し、付近に居た通行人ら7名が負傷する事件が発生した 


取得したドキュメント3: (Score: 75.18238067626953)
2月15日
オーストリアの都市フィラッハで、23歳のシリア人が居合わせた歩行者6人に対して無差別殺傷事件を起こす 


取得したドキュメント4: (Score: 75.08028411865234)
尚、アメリカ連邦捜査局（FBI）はニューオーリンズの事件とラスベガスの事件に関連性があるかどうかについて捜査を行う方針 


取得したドキュメント5: (Score: 73.02055358886719)
犯人は、42歳のアメリカ国籍の男で、警察官に向かって発砲し、銃撃戦により死亡した 


取得したドキュメント1: (Score: 85.48601531982422)
1月6日
カナダのジャスティン・トルドー首相がインフレ対応による支持率低下や、現時点で少数与党で次の選挙で敗退が見込まれることを理由に首相辞任を表明する 


取得したドキュメント2: (Score: 74.29981231689453)
ドナルド・トランプ大統領が2月4日から発動するとしていたカナダとメキシコの製品に対する25％の関税を1ヶ月延期すると表明 


取得したドキュメント3: (Score: 73.98201751708984)
1月4日 - オーストリアのカール・ネーハマー首相が2024年9月29日の国民議会 (オーストリア) 選挙での敗北とその後の連立政権樹立に向けた交渉の決裂により首相辞任を表明する 


取得したドキュメント4: (Score: 71.51438903808594

,Id,Question,Answer,withoutRAG,Query,withRAG
0,0,2025年1月1日にアメリカのニューオーリンズで発生した事件について、犯人の国籍、使用された...,2025年1月1日にアメリカ合衆国ルイジアナ州ニューオーリンズで発生した事件の犯人は42歳の...,申し訳ありませんが、2025年1月1日にニューオーリンズやラスベガスで発生した事件に関する情...,[参考資料]\n* 1月\n1月1日\nアメリカ合衆国ルイジアナ州ニューオーリンズで、新年を...,## 2025年1月1日アメリカのニューオーリンズ・ラスベガス事件について\n\n**ニュー...
1,1,2025年1月にカナダのジャスティン・トルドー首相が辞任を表明した理由は何ですか？,カナダのジャスティン・トルドー首相が2025年1月6日に辞任を表明した理由は、インフレ対応に...,2025年1月にカナダのジャスティン・トルドー首相が辞任を表明した理由は、**複雑な政治的状...,[参考資料]\n* 1月6日\nカナダのジャスティン・トルドー首相がインフレ対応による支持率...,2025年1月にカナダのジャスティン・トルドー首相が辞任を表明した理由は、**インフレ対応に...
2,2,2025年4月4日に韓国で憲法裁判所により罷免された大統領は誰ですか？,2025年4月4日に韓国で憲法裁判所により罷免された大統領は尹錫悦（ユン・ソンニョル）氏です。,2025年4月4日に韓国で憲法裁判所により罷免された大統領は **尹錫悦** です。 \n\...,[参考資料]\n* 4月4日 - 韓国の大統領・尹錫悦が憲法裁判所により罷免される\n* 罷...,2025年4月4日に韓国で憲法裁判所により罷免された大統領は **尹錫悦** です。 \n\...
3,3,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画はどのように変更さ...,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画は多額の投資へと変...,2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画は、**当初の計...,[参考資料]\n* この会談で日本製鉄のUSスチール買収計画が多額の投資へ変更になる\n* ...,参考資料によると、2025年2月7日に行われた日米首脳会談で、日本製鉄のUSスチール買収計画...
4,4,オーストリアのカール・ネーハマー首相が辞任を表明したのはいつですか？,オーストリアのカール・ネーハマー首相が辞任を表明したのは2025年1月4日です。,オーストリアのカール・ネーハマー首相が辞任を表明したのは **2023年10月17日** で...,[参考資料]\n* 1月4日 - オーストリアのカール・ネーハマー首相が2024年9月29日...,オーストリアのカール・ネーハマー首相が辞任を表明したのは **1月4日** です。 \n\n...


In [ ]:
# result_dfをcsvとして保存
result_df.to_csv("/content/rag-experiment/result.csv", index=False)

In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

0.0
